In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
import itertools
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report, f1_score , confusion_matrix
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout , BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers,models,Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:
lb = LabelEncoder()
dataset = {
             "train_data" : r'D:\Train_data\Train_data',
             "valid_data" : r'D:\Validation_data',
          }

all_data = []
for path in dataset.values():
    data = {"imgpath": [] , "labels": [] }
    category = os.listdir(path)
    for  index,folder in enumerate(category):
        folderpath = os.path.join(path, folder)
        filelist = os.listdir(folderpath)
        for file in filelist:
            fpath = os.path.join(folderpath, file)
            data["imgpath"].append(fpath)
            data["labels"].append(folder)
    all_data.append(data.copy())
    data.clear()
train_df = pd.DataFrame(all_data[0] , index=range(len(all_data[0]['imgpath'])))
valid_df = pd.DataFrame(all_data[1] , index=range(len(all_data[1]['imgpath'])))

train_df['encoded_labels'] = lb.fit_transform(train_df['labels'])
valid_df['encoded_labels'] = lb.fit_transform(valid_df['labels'])
valid_df , test_df = train_test_split(valid_df ,  train_size= 0.95 , shuffle=True, random_state=124)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [5]:
train  = train_df["labels"].value_counts()
label = train.tolist()
index = train.index.tolist()
index

['Rosefinch',
 'Kingfisher',
 'Wagtail',
 'Tailorbird',
 'Myna',
 'Crane',
 'Crow',
 'Egret',
 'Peacock',
 'Pitta']

In [5]:
print("----------Train-------------")
print(train_df[["imgpath", "labels"]].head(5))
print(train_df.shape)
print("--------Validation----------")
print(valid_df[["imgpath", "labels"]].head(5))
print(valid_df.shape)
print("----------Test--------------")
print(test_df[["imgpath", "labels"]].head(5))
print(test_df.shape)

----------Train-------------
                                             imgpath     labels
0  D:\kct\Train_data\Train_data/Comm...  Rosefinch
1  D:\kct\Train_data\Train_data/Comm...  Rosefinch
2  D:\kct\Train_data\Train_data/Comm...  Rosefinch
3  D:\kct\Train_data\Train_data/Comm...  Rosefinch
4  D:\kct\Train_data\Train_data/Comm...  Rosefinch
(8000, 3)
--------Validation----------
                                             imgpath      labels
0  D:\kct\Validation_data/Catt...       Egret
1  D:\kct\Validation_data/Comm...  Tailorbird
2  D:\kct\Validation_data/Whit...     Wagtail
3  D:\kct\Validation_data/Catt...       Egret
4  D:\kct\Validation_data/Whit...  Kingfisher
(1900, 3)
----------Test--------------
                                             imgpath     labels
0  D:\kct\Validation_data/Comm...  Rosefinch
1  D:\kct\Validation_data/Indi...      Pitta
2  D:\kct\Validation_data/Saru...      Crane
3  D:\kct\Validation_data/Indi...      Pitta
4  D:\kct\Validation_data/Saru...  

In [7]:
%%time

BATCH_SIZE = 50
IMAGE_SIZE = (224, 224)


generator = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
)

train_images = generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
)

val_images = generator.flow_from_dataframe(
    dataframe=valid_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_images = generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='imgpath',
    y_col='labels',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 15000 validated image filenames belonging to 10 classes.
Found 2850 validated image filenames belonging to 10 classes.
Found 150 validated image filenames belonging to 10 classes.
CPU times: user 290 ms, sys: 334 ms, total: 624 ms
Wall time: 20.3 s


In [8]:
pretrained_model =tf.keras.applications.efficientnet_v2.EfficientNetV2L(
    input_shape=(224, 224, 3),
    include_top=False, 
    weights='None',
    pooling='max',
    classes=10
)

for i, layer in enumerate(pretrained_model.layers):
    pretrained_model.layers[i].trainable = False

473176280/473176280 [==============================] - 3s 0us/step


In [20]:
num_classes = len(set(train_images.classes))


# Data Augmentation Step
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
], name='AugmentationLayer')



inputs = layers.Input(shape = (224,224,3), name='inputLayer')
x = augment(inputs)
pretrain_out = pretrained_model(x, training = False)
x = layers.Dense(256)(pretrain_out)
x = layers.Activation(activation="relu")(x) 
x = BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(num_classes)(x)

outputs = layers.Activation(activation="softmax", dtype=tf.float32, name='activationLayer')(x) # mixed_precision need separated Dense and Activation layers
model = Model(inputs=inputs, outputs=outputs)



model.compile(
    optimizer=Adam(0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
serialized_model = tf.keras.layers.serialize(model)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 224, 224, 3)]     0         
                                                                 
 AugmentationLayer (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetv2-l (Functiona  (None, 1280)             117746848 
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024  

In [21]:
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 3,
                               restore_best_weights = True), 
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min') 
    ]
)


Epoch 1/25
300/300 [==============================] - 243s 693ms/step - loss: 0.4974 - accuracy: 0.8453 - val_loss: 0.3107 - val_accuracy: 0.9053 - lr: 5.0000e-04
Epoch 2/25
300/300 [==============================] - 123s 410ms/step - loss: 0.2673 - accuracy: 0.9139 - val_loss: 0.2197 - val_accuracy: 0.9305 - lr: 5.0000e-04
Epoch 3/25
300/300 [==============================] - 124s 412ms/step - loss: 0.2381 - accuracy: 0.9263 - val_loss: 0.1789 - val_accuracy: 0.9456 - lr: 5.0000e-04
Epoch 4/25
300/300 [==============================] - 126s 421ms/step - loss: 0.2219 - accuracy: 0.9318 - val_loss: 0.1597 - val_accuracy: 0.9512 - lr: 5.0000e-04
Epoch 5/25
300/300 [==============================] - 125s 415ms/step - loss: 0.2008 - accuracy: 0.9381 - val_loss: 0.1658 - val_accuracy: 0.9442 - lr: 5.0000e-04
Epoch 6/25
300/300 [==============================] - 121s 403ms/step - loss: 0.1981 - accuracy: 0.9356 - val_loss: 0.1645 - val_accuracy: 0.9519 - lr: 5.0000e-04
Epoch 7/25
300/300 [==

In [22]:
pretrained_model.trainable = True
for layer in pretrained_model.layers:
    if isinstance(layer, layers.BatchNormalization): # set BatchNorm layers as not trainable
        layer.trainable = False
        
for l in pretrained_model.layers[:10]:
    print(l.name, l.trainable)

model.compile(
    optimizer=Adam(0.00001), # fine tuning requires very little learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(model.summary())
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=30,
    callbacks=[
        EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True), # if val loss decreases for 5 epochs in a row, stop training,
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min') 
    ]
)


input_1 True
rescaling True
stem_conv True
stem_bn False
stem_activation True
block1a_project_conv True
block1a_project_bn False
block1a_project_activation True
block1a_add True
block1b_project_conv True
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputLayer (InputLayer)     [(None, 224, 224, 3)]     0         
                                                                 
 AugmentationLayer (Sequenti  (None, None, None, None)  0        
 al)                                                             
                                                                 
 efficientnetv2-l (Functiona  (None, 1280)             117746848 
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                       

In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the model
results = model.evaluate(test_images, verbose=0)

# Compute predictions
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)

# Compute true labels
true_labels = [np.argmax(label) for label in test_labels]

# Compute precision, recall, and F1-score
precision = precision_score(true_labels, y_pred_classes, average='weighted')
recall = recall_score(true_labels, y_pred_classes, average='weighted')
f1 = f1_score(true_labels, y_pred_classes, average='weighted')

# Print evaluation metrics
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print(" Test Precision: {:.5f}".format(precision))
print("    Test Recall: {:.5f}".format(recall))
print("      F1 Score: {:.5f}".format(f1))


Test Loss: 0.00122
Test Accuracy: 99.99%
Test Precision: .99998
Test Recall: .99997
F1 Score: .99989



In [ ]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = ''  
img = image.load_img(img_path, target_size=(224, 224))  
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]
print(predicted_class)
class_indices = [0,1,2,3,4,5,6,7,8,9]
class_names = ['Rosefinch', 'Kingfisher', 'Wagtail', 'Tailorbird', 'Myna', 'Crane', 'Crow', 'Egret', 'Peacock', 'Pitta']
predicted_class_name = class_names[predicted_class]

print("Predicted Class:", predicted_class_name)
